# digit_recognition
Use a neural network to try to identify drawn digits.

In [1]:
import math

# The sigmoid transfer function.
def transfer_function(x):
    return 1 / (1 + math.exp(-x))

# The derivative of the sigmoid transfer function.
def transfer_derivative(output):
    return output * (1 - output)

In [2]:
class Neuron:
    def __init__(self, name):
        self.name = name
        self.inputs = []
        self.outputs = []
        self.sum = 1   # Set sum and value to 1 (mostly for bias neurons).
        self.output = 1

    # Use our input synapses to set our value.
    def set_value(self):
        # Don't change the bias neuron values.
        if self.name == 'Bias':
            return

        # Add up the inputs.
        self.sum = 0
        for synapse in self.inputs:
            self.sum += synapse.from_neuron.output * synapse.weight

        # Run through the transfer function.
        self.output = transfer_function(self.sum)

In [3]:
class Synapse:
    def __init__(self, from_neuron, to_neuron, weight):
        self.from_neuron = from_neuron
        self.to_neuron = to_neuron
        self.weight = weight

        if from_neuron is not None:  # This happens if this is a bias neuron.
            from_neuron.outputs.append(self)
        if to_neuron is not None:    # This should not happen.
            to_neuron.inputs.append(self)

In [4]:
import random

# layer_sizes is a list holding the number of
# neurons (not counting biases) in the layers.
class NeuralNet:
    def __init__(self, layer_sizes):
        # Build the layers.
        self.all_layers = []
        num_layers = len(layer_sizes)
        for i in range(num_layers):
            add_bias = i < num_layers - 1  # No bias for after the output layer.
            self.all_layers.append(self.build_layer(layer_sizes[i], f'Neuron_{i}', add_bias))

        # Make shortcuts to the input and output layers.
        self.input_layer = self.all_layers[0]
        self.output_layer = self.all_layers[-1]

        # Make synapses between layers.
        for layer_num in range(num_layers - 1):
            self.build_synapses(
                self.all_layers[layer_num],
                self.all_layers[layer_num + 1])

    # Make a list of {num_neurons} neurons.
    # Give them names {base_name}_{i} where {i} is the neuron number.
    # If {add_bias} is True, add a bias neuron.
    def build_layer(self, num_neurons, base_name, add_bias):
        layer = []

        # Create the neurons.
        for i in range(num_neurons):
            layer.append(Neuron(f'{base_name}_{i}'))

        # Add the bias neuron.
        if add_bias:
            layer.append(Neuron(f'Bias'))
        return layer

    # Make links connecting two neuron layers.
    def build_synapses(self, from_layer, to_layer):
        for from_neuron in from_layer:
            for to_neuron in to_layer:
                # No synapses lead into a bias neuron.
                if not to_neuron.name.startswith('Bias'):
                    Synapse(from_neuron, to_neuron, 0.5)

    # Run the network on these inputs.
    # Return the index of the most active output neuron
    # and a measure of confidence in that result.
    def evaluate(self, input_values):
        # Make sure we have the right number of input values.
        # (The +1 is for the bias neuron in the input layer.)
        if len(input_values) + 1 != len(self.input_layer):
            raise TypeError('Number of input values does not match the size of the input layer')

        # Set the input layer values.
        for i in range(len(input_values)):
            self.input_layer[i].output = input_values[i]

        # Update the network.
        for i in range(1, len(self.all_layers)):
            # Update layer i.
            for neuron in self.all_layers[i]:
                neuron.set_value()

        # See which output neuron is most active and
        # calculate the confidence value.
        outputs = [neuron.output for neuron in self.output_layer]
        best_output, best_i = max((value, index) for index, value in enumerate(outputs))
        total_output = sum(output for output in outputs if output > 0.5)
        if total_output == 0:
            total_output = 1

        return best_i, best_output / total_output

    # Print the node values.
    def dump(self):
        for layer_num in range(len(self.all_layers)):
            print(f'Layer {layer_num}:')
            for neuron in self.all_layers[layer_num]:
                print(f'    {neuron.name}: ', end='')
                for synapse in neuron.outputs:
                    print(f'{synapse.weight:>7.2f} ', end='')
                print()

    # Randomize the network's synapse weights.
    def randomize(self, min_weight, max_weight):
        for layer in self.all_layers:
            for neuron in layer:
                for synapse in neuron.inputs:
                    synapse.weight = random.uniform(min_weight, max_weight)

    # Train the network for one epoch.
    # {input_list} is a list of sets of inputs.
    # {target_list} is a list holding corresponding sets of target outputs.
    # Return the average error over this epoch.
    def train_one_epoch(self, input_list, target_list, learning_rate):
        # Randomize the input and target lists.
        combined = list(zip(input_list, target_list))
        random.shuffle(combined)
        input_list, target_list = zip(*combined)

        # Process the epoch.
        total_error = 0
        for i in range(len(input_list)):
            # Evaluate this input.
            self.evaluate(input_list[i])

            # Use back propagation to adjust synapse values.
            total_error += self.back_propagate(target_list[i], learning_rate)

        return total_error / len(input_list)

    # Train the network for {num_epochs} epochs.
    def train(self, inputs, targets, learning_rate, num_epochs):
        for i in range(num_epochs):
            epoch_error = self.train_one_epoch(inputs, targets, learning_rate)
        return epoch_error

    # Calculate output neuron errors.
    # Use the half square error formula.
    def calculate_error(self, targets):
        total_error = 0

        # Loop through the output layer's synapses by index.
        for i in range(len(self.output_layer)):
            # Get the neuron by index.
            neuron = self.output_layer[i]

            # Calculate the difference between the
            # synapse's output and the target output.
            diff = (targets[i] - neuron.output)

            # Add to the total error.
            total_error += diff * diff
        return total_error / len(self.output_layer)

    # Use back propagation to adjust synapse values.
    # https://machinelearningmastery.com/implement-backpropagation-algorithm-scratch-python/
    # In the article's terms, 'net' is a neuron's total input.
    # Return the total error.
    def back_propagate(self, targets, learning_rate):
        DEBUG = False
        if DEBUG: print('**********************\n*** back_propagate ***\n**********************')

        # Calculate output neuron errors.
        total_error = self.calculate_error(targets)

        # Calculate de_do and de_di.
        for i in range(len(self.output_layer)):
            neuron = self.output_layer[i]
            neuron.de_do = -(targets[i] - neuron.output)
            if DEBUG: print(f'de_do {i}: {neuron.de_do}')
            neuron.do_di = neuron.output * (1 - neuron.output)
            if DEBUG: print(f'do_di {i}: {neuron.do_di}')

        # Calculate the changes from the synapses leading to the output layer.
        if DEBUG: print('Output synapses:')
        for neuron in self.output_layer:
            for synapse in neuron.inputs:
                synapse.dnet_dw = synapse.from_neuron.output
                if DEBUG: print(f'    {synapse.from_neuron.name} --> {synapse.to_neuron.name}:')
                if DEBUG: print(f'        dnet_dw: {synapse.dnet_dw}')
                synapse.de_dw = synapse.to_neuron.de_do * synapse.to_neuron.do_di * synapse.dnet_dw
                if DEBUG: print(f'        de_dw:   {synapse.de_dw}')

                # Calculate the synapse's new weight.
                synapse.new_weight = synapse.weight - learning_rate * synapse.de_dw
                if DEBUG: print(f'        new weight:  {synapse.new_weight}')
        if DEBUG: print()

        # Calculate the changes from the synapses leading to the hidden layer.
        if DEBUG: print('Hidden synapses:')
        for neuron in self.all_layers[1]:
            neuron.de_do = 0
            neuron.de_dw = 0
            neuron.do_di = neuron.output * (1 - neuron.output)
            for synapse in neuron.outputs:
                if DEBUG: print(f'    {neuron.name} --> {synapse.to_neuron.name}:')
                synapse.de_do = synapse.to_neuron.de_do * synapse.to_neuron.do_di
                if DEBUG: print(f'        de_do = {synapse.to_neuron.de_do} * {synapse.to_neuron.do_di} = {synapse.to_neuron.de_do * synapse.to_neuron.do_di}')
                synapse.do_di = synapse.de_do * synapse.weight
                if DEBUG: print(f'        de_di = {synapse.de_do} * {synapse.weight} = {synapse.de_do * synapse.weight}')
                neuron.de_do += synapse.do_di
            if DEBUG: print(f'    Neuron {neuron.name}')
            if DEBUG: print(f'        de_do: {neuron.de_do}')
            if DEBUG: print(f'        do_di: {neuron.do_di}')

            neuron.de_di = 0
            for synapse in neuron.inputs:
                if DEBUG: print(f'    {synapse.from_neuron.name} --> {synapse.to_neuron.name}:')
                synapse.di_dw = synapse.from_neuron.output
                if DEBUG: print(f'        di_dw: {synapse.di_dw}')
                synapse.de_dw = neuron.de_do * neuron.do_di * synapse.di_dw
                if DEBUG: print(f'        synapse.de_dw = {neuron.de_do} * {neuron.do_di} * {synapse.di_dw} = {synapse.de_dw} AAA')

                synapse.new_weight = synapse.weight - learning_rate * synapse.de_dw
                if DEBUG: print(f'        synapse.new_weight = {synapse.weight} - {learning_rate} * {synapse.de_dw} = {synapse.new_weight} BBB')
                synapse.new_weight = synapse.weight - learning_rate * synapse.de_dw

        # Update all of the synapse weights.
        for layer in self.all_layers:
            for neuron in layer:
                for synapse in neuron.outputs:
                    synapse.weight = synapse.new_weight

        return total_error

In [5]:
class DataPoint:
    # data_string is a string holding the digit, 0s, and 1s
    # in the format '6: 011110110000100000111111110001110001010001001111'
    def __init__(self, data_string):
        fields = data_string.split(' ')
        self.name = fields[0][0]
        self.zeros_and_ones = fields[1].strip()

        # Make the properties list, which forms
        # the input for the neural network.
        self.properties = []
        for ch in self.zeros_and_ones:
            self.properties.append(int(ch))

        # Set the target list for the neural network.
        self.target = []
        for i in range(10):
            if i == int(self.name):
                self.target.append(1)
            else:
                self.target.append(0)

In [6]:
# Functions to draw the network.
NEURON_RADIUS = 4

# Position the neurons for drawing.
def position_neurons(network, canvas):
    # Get the canvas's dimensions.
    canvas.winfo_toplevel().update()
    width = canvas.winfo_width()
    height = canvas.winfo_height()

    # Find the maximum number of neurons in any layer.
    max_neurons = len(max(network.all_layers, key=len))

    # Calculate some spacing values.
    margin = NEURON_RADIUS + 5
    neuron_gap = (height - 2 * margin) / (max_neurons - 1)  # Spacing between neurons in a layer.
    layer_gap = (width - 2 * margin) / (len(network.all_layers) - 1)  # Spacing between layers.

    # Position the neurons.
    x = margin
    for layer in network.all_layers:
        neuron_gap = (height - 2 * margin) / (len(layer) - 1)  # Spacing between neurons in a layer.
        y = (height - neuron_gap * (len(layer) - 1)) / 2
        for neuron in layer:
            neuron.x = x
            neuron.y = y
            y += neuron_gap
        x += layer_gap

# Draw the network's synapses.
def draw_synapses(network, canvas):
    # Inputs.
    for layer in network.all_layers:
        for neuron_num in range(len(layer)):
            from_neuron = layer[neuron_num]
            for synapse in from_neuron.outputs:
                draw_synapse(synapse, canvas, neuron_num)

# Draw a synapse.
def draw_synapse(synapse, canvas, neuron_num):
    from_neuron = synapse.from_neuron
    to_neuron = synapse.to_neuron

    # Draw the line.
    canvas.create_line(
        from_neuron.x, from_neuron.y,
        to_neuron.x, to_neuron.y,
        fill='blue')

    # Skip the following; it's too much text to read anyway.
    if False:
        # Draw the weight text.
        x = (5 * from_neuron.x + 1 * to_neuron.x) / 6
        y = (5 * from_neuron.y + 1 * to_neuron.y) / 6
        text_tag = canvas.create_text(x, y, text=f'{synapse.weight:.2f}')

        # Clear an area for the text.
        bounds = canvas.bbox(text_tag)  # Returns (x1, y1, x2, y2)
        canvas.create_oval(*bounds, fill='white', outline='white')

        # Lift the label above the cleared oval.
        canvas.tag_raise(text_tag)

# Draw the network's neurons.
def draw_neurons(network, canvas):
    for layer in network.all_layers:
        for neuron in layer:
            draw_neuron(neuron, canvas)

# Draw a neuron.
def draw_neuron(neuron, canvas):
    width = 1
    if neuron.name.startswith('Bias'):
        width = 3

    color = 'silver'  # For uncomitted neurons.
    if neuron.name.startswith('Bias'):
        color = 'pink'
    elif neuron.output < 0.33:
        color = 'white'
    elif neuron.output > 0.67:
        color = 'lightgreen'

    canvas.create_oval(
        neuron.x - NEURON_RADIUS, neuron.y - NEURON_RADIUS,
        neuron.x + NEURON_RADIUS, neuron.y + NEURON_RADIUS,
        fill=color, outline='black', width=width)

# Make a simple drawing of the network.
def draw_network(network, canvas):
    canvas.delete('all')

    # Position the neurons.
    position_neurons(network, canvas)

    # Draw the synapses.
    draw_synapses(network, canvas)

    # Draw the neurons.
    draw_neurons(network, canvas)

In [7]:
# Create a label and an entry widget. Return the attached StringVar.
def make_entry(window, label_text, label_width, default_text, pady):
    frame = tk.Frame(window)
    frame.pack(side=tk.TOP, pady=pady)

    label = tk.Label(frame, text=label_text, width=label_width, anchor=tk.W)
    label.pack(side=tk.LEFT)

    string_var = tk.StringVar()
    string_var.set(default_text)
    entry = tk.Entry(frame, width=7,
        textvariable=string_var, justify=tk.RIGHT)
    entry.pack(side=tk.LEFT)

    return string_var

# Create a label and an output label. Return the attached StringVar.
def make_label(window, label_text, label_width, result_width, default_text, pady):
    frame = tk.Frame(window)
    frame.pack(side=tk.TOP, pady=pady)

    label = tk.Label(frame, text=label_text, width=label_width, anchor=tk.W)
    label.pack(side=tk.LEFT)

    result_label = tk.Label(frame, width=result_width)
    result_label['text'] = default_text
    result_label.pack(side=tk.LEFT)

    return result_label

In [8]:
import tkinter as tk

# Geometry constants.
WINDOW_WID = 910
WINDOW_HGT = 610
FRAME1_WID = 150
PADX = 5
NETWORK_CANVAS_WID = WINDOW_WID - FRAME1_WID - 4 * PADX
NETWORK_CANVAS_HGT = WINDOW_HGT - 2 * PADX
NUM_ROWS = 8
NUM_COLS = 6
CELL_WID = 20
CELL_HGT = CELL_WID
MARGIN = 5

class App:
    # Create and manage the tkinter interface.
    def __init__(self):
        self.network = None

        # Make the main interface.
        self.window = tk.Tk()
        self.window.title('digit_identification')
        self.window.protocol('WM_DELETE_WINDOW', self.kill_callback)
        self.window.geometry(f'{WINDOW_WID}x{WINDOW_HGT}')

        # Initially we have nothing to draw.
        self.polyline = None
        self.points = []

        # Build the UI.
        self.build_ui()

        # Load the data.
        self.load_data()

        # Display the window.
        self.window.focus_force()
        self.window.mainloop()

    def load_data(self):
        # Load the DataPoints.
        with open('digit_data.txt', 'r') as f:
            lines = f.readlines()
            self.data_points = []
            for line in lines:
                self.data_points.append(DataPoint(line))

    def redraw(self):
        # Remove old polyline.
        self.grid_canvas.delete(self.polyline)
        self.polyline = None

        # Draw current points.
        if len(self.points) > 1:
            self.polyline = self.grid_canvas.create_line(self.points, fill='black')

    def build_ui(self):
        # Make controls to define the network.
        frame1 = tk.Frame(self.window, width=FRAME1_WID)
        frame1.pack(side=tk.LEFT, expand=False, fill=tk.Y, padx=PADX)
        frame1.pack_propagate(False)

        frame2 = tk.Frame(self.window)
        frame2.pack(side=tk.LEFT, expand=True, fill=tk.BOTH, padx=PADX)

        # Number of neurons in the hidden layer.
        self.num_hidden_neurons = make_entry(frame1, 'Hidden Neurons:', 13, '29', (0,5))

        # Build button.
        build_button = tk.Button(frame1, width=11, text='Build', command=self.build)
        build_button.pack(side=tk.TOP, pady=(2,5))

        # Train button.
        self.train_button = tk.Button(frame1, width=11, text='Train', command=self.train)
        self.train_button.pack(side=tk.TOP, pady=(2,5))

        # Success Listbox.
        success_label = tk.Label(frame1, text='Success Rate:', width=12, anchor=tk.W)
        success_label.pack(side=tk.TOP)

        success_frame = tk.Frame(frame1)
        success_frame.pack(side=tk.TOP)
        scrollbar = tk.Scrollbar(success_frame, orient="vertical")
        self.success_listbox = tk.Listbox(success_frame, width=20, yscrollcommand=scrollbar.set)
        self.success_listbox.pack(side=tk.LEFT)
        scrollbar.config(command=self.success_listbox.yview)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        # Make the drawing canvas.
        canvas_wid = NUM_COLS * CELL_WID + 1
        canvas_hgt = NUM_ROWS * CELL_HGT + 1
        self.grid_canvas = tk.Canvas(frame1, bg='white',
            borderwidth=0, highlightthickness=0, relief=tk.SUNKEN, width=canvas_wid, height=canvas_hgt)
        self.grid_canvas.pack(side=tk.TOP, pady=5)
        self.grid_canvas.bind('<Button-1>', self.start_draw)
        self.grid_canvas.bind('<ButtonRelease-1>', self.end_draw)

        # Make grid lines.
        for r in range(NUM_ROWS + 1):
            self.grid_canvas.create_line(0, r * CELL_HGT, canvas_wid, r * CELL_HGT, fill='lime')
        for c in range(NUM_COLS + 1):
            self.grid_canvas.create_line(c * CELL_WID, 0, c * CELL_WID, canvas_hgt, fill='lime')

        # Make a big label to display results from the user drawing.
        self.user_result_var = tk.StringVar()
        self.user_result_label = tk.Label(frame1, font=('Calibri 80 normal'),
            textvariable=self.user_result_var)
        self.user_result_label.pack(side=tk.TOP)

        # Make a label to display confidence in the user drawing result.
        self.confidence_var = tk.StringVar()
        self.confidence_label = tk.Label(frame1, textvariable=self.confidence_var)
        self.confidence_label.pack(side=tk.TOP)

        # Frame 2.
        # Network canvas.
        self.network_canvas = tk.Canvas(frame2, bg='white',
            borderwidth=0, highlightthickness=0, relief=tk.SUNKEN,
            width=NETWORK_CANVAS_WID, height=NETWORK_CANVAS_HGT)
        self.network_canvas.pack(side=tk.LEFT, anchor=tk.NW)

    # Train the network.
    def train(self):
        if self.train_button['text'] == 'Stop':
            # Stop training.
            self.train_button.configure(text='Train')
            self.training = False
        else:
            # Start training.
            self.train_button.configure(text='Stop')

            # Set parameters.
            epochs_per_tick = 10
            learning_rate = 0.5

            # Build the inputs and targets lists.
            inputs = [data_point.properties for data_point in self.data_points]
            targets = [data_point.target for data_point in self.data_points]

            # Start training.
            self.training = True
            self.tick(inputs, targets, learning_rate, epochs_per_tick)

    # Train for {epochs_per_tick} epochs.
    def tick(self, inputs, targets, learning_rate, epochs_per_tick):
        if not self.training:
            return

        # Train.
        epoch_error = self.network.train(inputs, targets, learning_rate, epochs_per_tick)
        
        # Display the current success rate.
        self.num_epochs_trained += epochs_per_tick
        success_rate = self.calculate_success_rate(inputs, targets)
        self.success_listbox.insert(tk.END, f'{self.num_epochs_trained}: {success_rate:.6f}%')
        self.success_listbox.see(tk.END)

        # Repeat.
        TICK_MS = 10
        self.window.after(TICK_MS, self.tick, inputs, targets, learning_rate, epochs_per_tick)

    # Calculate the network's current success rate percentage.
    def calculate_success_rate(self, inputs, targets):
        num_successes = 0
        num_inputs = len(inputs)
        for i in range(num_inputs):
            digit, confidence = self.network.evaluate(inputs[i])
            if targets[i][digit] == 1:
                num_successes += 1
        return num_successes / num_inputs * 100

    def start_draw(self, event):
        # Clear any previous result.
        self.user_result_var.set('')
        self.confidence_var.set('')

        # Remove any previous drawing.
        self.points = []
        self.redraw()

        self.grid_canvas.bind('<B1-Motion>', self.save_point)

    def end_draw(self, event):
        self.grid_canvas.unbind('<B1-Motion>')

        # Evaluate the polyline.
        self.evaluate_polyline()

    def save_point(self, event):
        self.points.append((event.x, event.y))
        self.redraw()

    def kill_callback(self):
        self.window.destroy()

    # Build the network.
    def build(self):
        # Clear any previous results.
        self.success_listbox.delete(0, tk.END)
        self.user_result_var.set('')
        self.confidence_var.set('')
        self.points = []
        self.redraw()
        self.num_epochs_trained = 0

        # Build and display the network.
        num_hidden_neurons = int(self.num_hidden_neurons.get())
        self.network = NeuralNet([48, num_hidden_neurons, 10])

        # Randomize the network.
        MAX_WEIGHT = 1
        MIN_WEIGHT = -MAX_WEIGHT
        self.network.randomize(MIN_WEIGHT, MAX_WEIGHT)
        self.redraw_network()

    # Redraw the network after the user changed a checkbutton.
    def redraw_network(self):
        # Draw the network.
        draw_network(self.network, self.network_canvas)

    # See which group is closest to the polyline.
    def evaluate_polyline(self):
        # Convert the polyline into a DataPoint.
        data_point = self.polyline_to_data_point()

        # Run the neural network.
        digit, confidence = self.network.evaluate(data_point.properties)

        # Display the result.
        self.user_result_var.set(f'{digit}')

        # Display our confidence.
        self.confidence_var.set(f'Score: {confidence:.2f}')

    # Convert the polyline into a DataPoint.
    def polyline_to_data_point(self):
        # Convert the points into the cells that were touched.
        touched = self.get_touched()

        # Convert the touched cells to a string.
        touched_string = self.touched_to_string(touched)

        # Compose the DataPoint data string.
        data_string = f'0: {touched_string}'

        # Create the DataPoint.
        return DataPoint(data_string)

    # Convert the points into the cells that were touched.
    def get_touched(self):
        # Make a touched array holding 0s.
        touched = []
        for r in range(NUM_ROWS):
            touched.append([0 for i in range(NUM_COLS)])

        # Mark the touched cells.
        for point in self.points:
            r = int(point[1] / CELL_HGT)
            c = int(point[0] / CELL_WID)
            if r >= 0 and r < NUM_ROWS and c >= 0 and c < NUM_COLS:
                touched[r][c] = 1

        # Return the touched list.
        return touched

    # Return a string holding the touch values.
    def touched_to_string(self, touched):
        result = ''
        for r in range(NUM_ROWS):
            for c in range(NUM_COLS):
                result += str(touched[r][c])
        return result

In [ ]:
App()